# [**Stratégie personnalisée de réajustement d'une recherche en grille avec validation croisée**](https://nbviewer.org/github/Franck-PepperLabs/pepper_dsia_skl_doc_fr/blob/main/docs/examples/3_model_selection/plot_grid_search_digits.ipynb)<br/>([_Custom refit strategy of a grid search with cross-validation_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html))

# Stratégie personnalisée de réajustement d'une recherche en grille avec validation croisée

Cet exemple montre comment un classifieur est optimisé par validation croisée, en utilisant l'objet [**GridSearchCV**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) sur un ensemble de développement qui ne comprend que la moitié des données étiquetées disponibles.

Les performances des hyperparamètres sélectionnés et du modèle entraîné sont ensuite mesurées sur un ensemble d'évaluation dédié qui n'a pas été utilisé lors de l'étape de sélection du modèle.

Les outils disponibles pour la sélection de modèle sont détaillés dans les sections sur [**Validation croisée : évaluation des performances de l'estimateur** (3.1)](https://scikit-learn.org/stable/modules/cross_validation.html) et [**Ajustement des hyperparamètres d'un estimateur** (3.2)](https://scikit-learn.org/stable/modules/grid_search.html).

# Le jeu de données

Nous travaillerons avec l'ensemble de données des chiffres manuscrits. L'objectif est de classer des images de chiffres écrits à la main. Nous transformons le problème en une classification binaire pour une meilleure compréhension : l'objectif est d'identifier si un chiffre est `8` ou non.

In [2]:
from sklearn import datasets

digits = datasets.load_digits()

Pour entraîner un classifieur sur des images, nous devons les aplatir en vecteurs. Chaque image de 8 par 8 pixels doit être transformée en un vecteur de 64 pixels. Nous obtiendrons ainsi un tableau de données final de forme `(n_images, n_pixels)`.

In [3]:
n_samples = len(digits.images)
X = digits.images.reshape((n_samples, -1))
y = digits.target == 8
print(
    f"The number of images is {X.shape[0]} and each image contains {X.shape[1]} pixels"
)

The number of images is 1797 and each image contains 64 pixels


Comme présenté dans l'introduction, les données seront divisées en un ensemble d'entraînement et un ensemble de test de même taille.

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

# Définir notre stratégie de recherche en grille

Nous sélectionnerons un classifieur en recherchant les meilleurs hyperparamètres sur les plis de l'ensemble d'entraînement. Pour ce faire, nous devons définir les scores pour sélectionner le meilleur candidat.

In [5]:
scores = ["precision", "recall"]

Nous pouvons également définir une fonction à passer au paramètre `refit` de l'instance [**GridSearchCV**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Elle mettra en œuvre la stratégie personnalisée pour sélectionner le meilleur candidat à partir de l'attribut `cv_results_` de [**GridSearchCV**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Une fois le candidat sélectionné, il est automatiquement réajusté par l'instance [**GridSearchCV**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).

Ici, la stratégie consiste à présélectionner les meilleurs modèles en termes de précision et de rappel. Parmi les modèles sélectionnés, nous choisissons finalement le modèle le plus rapide en termes de prédiction. Remarquez que ces choix personnalisés sont totalement arbitraires.

In [6]:
import pandas as pd


def print_dataframe(filtered_cv_results):
    """Pretty print for filtered dataframe"""
    for mean_precision, std_precision, mean_recall, std_recall, params in zip(
        filtered_cv_results["mean_test_precision"],
        filtered_cv_results["std_test_precision"],
        filtered_cv_results["mean_test_recall"],
        filtered_cv_results["std_test_recall"],
        filtered_cv_results["params"],
    ):
        print(
            f"precision: {mean_precision:0.3f} (±{std_precision:0.03f}),"
            f" recall: {mean_recall:0.3f} (±{std_recall:0.03f}),"
            f" for {params}"
        )
    print()


def refit_strategy(cv_results):
    """Define the strategy to select the best estimator.

    The strategy defined here is to filter-out all results below a precision threshold
    of 0.98, rank the remaining by recall and keep all models with one standard
    deviation of the best by recall. Once these models are selected, we can select the
    fastest model to predict.

    Parameters
    ----------
    cv_results : dict of numpy (masked) ndarrays
        CV results as returned by the `GridSearchCV`.

    Returns
    -------
    best_index : int
        The index of the best estimator as it appears in `cv_results`.
    """
    # print the info about the grid-search for the different scores
    precision_threshold = 0.98

    cv_results_ = pd.DataFrame(cv_results)
    print("All grid-search results:")
    print_dataframe(cv_results_)

    # Filter-out all results below the threshold
    high_precision_cv_results = cv_results_[
        cv_results_["mean_test_precision"] > precision_threshold
    ]

    print(f"Models with a precision higher than {precision_threshold}:")
    print_dataframe(high_precision_cv_results)

    high_precision_cv_results = high_precision_cv_results[
        [
            "mean_score_time",
            "mean_test_recall",
            "std_test_recall",
            "mean_test_precision",
            "std_test_precision",
            "rank_test_recall",
            "rank_test_precision",
            "params",
        ]
    ]

    # Select the most performant models in terms of recall
    # (within 1 sigma from the best)
    best_recall_std = high_precision_cv_results["mean_test_recall"].std()
    best_recall = high_precision_cv_results["mean_test_recall"].max()
    best_recall_threshold = best_recall - best_recall_std

    high_recall_cv_results = high_precision_cv_results[
        high_precision_cv_results["mean_test_recall"] > best_recall_threshold
    ]
    print(
        "Out of the previously selected high precision models, we keep all the\n"
        "the models within one standard deviation of the highest recall model:"
    )
    print_dataframe(high_recall_cv_results)

    # From the best candidates, select the fastest model to predict
    fastest_top_recall_high_precision_index = high_recall_cv_results[
        "mean_score_time"
    ].idxmin()

    print(
        "\nThe selected final model is the fastest to predict out of the previously\n"
        "selected subset of best models based on precision and recall.\n"
        "Its scoring time is:\n\n"
        f"{high_recall_cv_results.loc[fastest_top_recall_high_precision_index]}"
    )

    return fastest_top_recall_high_precision_index

# Ajustement des hyperparamètres

Une fois que nous avons défini notre stratégie pour sélectionner le meilleur modèle, nous définissons les valeurs des hyperparamètres et créons l'instance de recherche en grille :

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

tuned_parameters = [
    {"kernel": ["rbf"], "gamma": [1e-3, 1e-4], "C": [1, 10, 100, 1000]},
    {"kernel": ["linear"], "C": [1, 10, 100, 1000]},
]

grid_search = GridSearchCV(
    SVC(), tuned_parameters, scoring=scores, refit=refit_strategy
)
grid_search.fit(X_train, y_train)

All grid-search results:
precision: 1.000 (±0.000), recall: 0.854 (±0.063), for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
precision: 1.000 (±0.000), recall: 0.257 (±0.061), for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
precision: 1.000 (±0.000), recall: 0.877 (±0.069), for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
precision: 0.968 (±0.039), recall: 0.780 (±0.083), for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
precision: 1.000 (±0.000), recall: 0.877 (±0.069), for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
precision: 0.905 (±0.058), recall: 0.889 (±0.074), for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
precision: 1.000 (±0.000), recall: 0.877 (±0.069), for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
precision: 0.904 (±0.058), recall: 0.890 (±0.073), for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
precision: 0.695 (±0.073), recall: 0.743 (±0.065), for {'C': 1, 'kernel': 'linear'}
precision: 0.643 (±0.066), recall: 0.757 (±0.066), for {'C': 10, 'kernel': 'linear'}
precision: 0.6

GridSearchCV(estimator=SVC(),
             param_grid=[{'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']},
                         {'C': [1, 10, 100, 1000], 'kernel': ['linear']}],
             refit=<function refit_strategy at 0x000002362A54E0C0>,
             scoring=['precision', 'recall'])

Les paramètres sélectionnés par la recherche en grille avec notre stratégie personnalisée sont :

In [8]:
grid_search.best_params_

{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}

Enfin, nous évaluons le modèle affiné sur l'ensemble d'évaluation mise en réserve : l'objet `grid_search` **a été automatiquement réajusté** sur l'ensemble d'entraînement complet avec les paramètres sélectionnés par notre stratégie personnalisée de réajustement.

Nous pouvons utiliser le rapport de classification pour calculer les métriques standards de classification sur l'ensemble mis en réserve :

In [9]:
from sklearn.metrics import classification_report

y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.99      1.00      0.99       807
        True       1.00      0.87      0.93        92

    accuracy                           0.99       899
   macro avg       0.99      0.93      0.96       899
weighted avg       0.99      0.99      0.99       899



> **Note** : Le problème est trop facile : le plateau des hyperparamètres est trop plat et le modèle de sortie est le même en termes de précision et de rappel, et de qualité.